In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
import numpy as np
import pandas as pd
import json
from PIL import Image
import os

from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import cv2
from scipy.special import softmax

hyperparams = {
    "BATCH": 32,
    "EPOCHS": 32,
    "LR": 0.0001,
    "model_name": 'resnext50_32x4d'
}
IM_SIZE = 512
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
print(DEVICE)

labels = json.load(open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"))
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

X_Train, Y_Train = train['image_id'].values, train['label'].values
X_Test = [name for name in (os.listdir(TEST_DIR))]

class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, index):
        image = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:
            return self.transform(image), self.lbs[index]
        elif "test" in self.dir:
            return self.transform(image), self.fnames[index]

Transform = transforms.Compose(
    [transforms.RandomResizedCrop(224),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)

class CustomResnet(nn.Module):
    def __init__(self, model_name = 'skresnext50_32x4d', pretrained = False):
        super().__init__()
        self.model = timm.create_model(model_name = model_name, pretrained = pretrained)
        n_features = self.model.fc.in_features
        self.model.fc == nn.Linear(n_features, 5, bias = True)
    def forward(self, x):
        return self.model(x)

class enet_v2(nn.Module):
    def __init__(self, backbone = 'efficientnet_b3a', outdim = 5, pretrained=False):
        super(enet_v2, self).__init__()
        self.enet = timm.create_model(backbone, pretrained = pretrained)
        in_ch = self.enet.classifier.in_features
        self.myfc = nn.Linear(in_ch, outdim)
        self.enet.classifier = nn.Identity()
        
    def forward(self, x):
        x = self.enet(x)
        x = self.myfc(x)
        return x

model = CustomResnet(pretrained = False)
model = model.to(DEVICE)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = hyperparams["LR"])
learning_rate_scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

def finetune(model, trainloader, optimizer, learning_rate_scheduler, model_name):
    model.train()
    train_loss = 0.0
    for i, (images, labels) in enumerate(tqdm(trainloader)):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        train_result = model(images)
        loss = loss_func(train_result, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.detach().item()
        del labels
        del images
        torch.cuda.empty_cache()
    learning_rate_scheduler.step()
    if(model_name=="res"):
        torch.save(model.state_dict(), './checkpoints/'+'res.pt')
    else:
        torch.save(model.state_dict(), './checkpoints/'+'eff.pt')
    print('Train Loss: %.4f'%train_loss)

model2 = enet_v2(pretrained = False)
model2 = model2.to(DEVICE)
optimizer2 = torch.optim.Adam(model2.parameters(), lr = hyperparams["LR"])
learning_rate_scheduler = lr_scheduler.ExponentialLR(optimizer2, gamma=0.99)

def train_res(pretrained = True):
    trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
    trainloader = DataLoader(trainset, batch_size = hyperparams["BATCH"], shuffle=True, num_workers=4)
    os.makedirs('./checkpoints', exist_ok = True)
    for epoch in range(hyperparams["EPOCHS"]):
        print('******* EPOCH %d / %d ********' % (epoch+1, hyperparams['EPOCHS']))
        finetune(model, trainloader, optimizer, learning_rate_scheduler, "res")

    print("resnet train finished")

def train_eff():
    trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
    trainloader = DataLoader(trainset, batch_size = hyperparams["BATCH"], shuffle=True, num_workers=4)
    os.makedirs('./checkpoints', exist_ok = True)
    for epoch in range(hyperparams["EPOCHS"]):
         print('******* EPOCH %d / %d ********' % (epoch+1, hyperparams['EPOCHS']))
         finetune(model2, trainloader, optimizer2, learning_rate_scheduler, "eff")

    print("effnet train finished")

# train_res()
# train_eff()

######################infer########################

def infer():
    testset = GetData(TEST_DIR, X_Test, None, Transform)
    testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)
    res_preds = []
    eff_preds = []
    model.load_state_dict(torch.load('../input/checkpoint5/res.pt', map_location = DEVICE))
    model2.load_state_dict(torch.load('../input/checkpoint6/eff.pt', map_location = DEVICE))
    model.eval()

    files = []

    with torch.no_grad():
        for image, fname in tqdm(testloader):
            image = image.to(DEVICE)
            logits = model(image)
            ps = torch.exp(logits)
            res_preds += [torch.softmax(logits, 1).detach().cpu()]
            files.append(fname[0])
        res_preds = torch.cat(res_preds).cpu().numpy()
    model2.eval()
    with torch.no_grad():
        for image, fname in tqdm(testloader):
            x = image.to(DEVICE)
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
            x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
            x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],0)
            x = x.view(-1, 3, IM_SIZE, IM_SIZE)
            logits = model(x)
            logits = logits.view(1, 8, -1).mean(1)
            eff_preds += [torch.softmax(logits, 1).detach().cpu()]

        eff_preds = torch.cat(eff_preds).cpu().numpy()
    pred = 0.5*res_preds + 0.5 *eff_preds
    
    return pred, files

fin_pred, files = infer()
s_ls = []
fin = softmax(fin_pred).argmax(1)

for i in range(len(fin_pred)):
    s_ls.append([files[i], fin[i]])

sub = pd.DataFrame.from_records(s_ls, columns=['image_id', 'label'])
sub.to_csv("submission.csv", index = False)

In [ ]:
sub